In [1]:
import altair as alt
import ipywidgets as widgets
from ipywidgets import interactive, interact, HBox, Layout, VBox, interact_manual
from IPython.display import display

# Simple Stacked Area Chart


In [2]:
# replace the url with the url to your data
#url = "https://wri-rw.carto.com/api/v2/sql?q=SELECT origin AS c, date::text AS x, total AS y FROM blog_002a_asylum_requests_country_of_origin WHERE origin IN ('Mexico','Guatemala','China','El Salvador','Honduras','Haiti','India','Colombia', 'Ecuador','Venezuela (Bolivarian Republic of)') AND EXTRACT(YEAR FROM date) < 2018 ORDER BY origin ASC, date ASC"
#url = "https://wri-rw.carto.com/api/v2/sql?q=SELECT year AS x, value AS y, production_or_use AS c FROM foo_050_fish_capture_and_production WHERE production_or_use IN ('Inland Aquaculture','Marine Aquaculture','Inland Capture','Marine Capture') ORDER BY year ASC, production_or_use!='Marine Capture', production_or_use!='Inland Capture', production_or_use!='Marine Aquaculture', production_or_use!='Inland Aquaculture'"
url = "https://wri-rw.carto.com/api/v2/sql?q=SELECT datetime as x, yr_data AS y, sector AS c FROM cli_049_rw1_dash_pik_historical_emissions_edit WHERE sector NOT ILIKE 'Total excluding LULUCF'"

data = alt.UrlData(url, format = alt.JsonDataFormat(property='rows'))

In [3]:
# run the function 
def make_stacked_area_chart(x_label, x_variable, x_datatype, x_zero, x_grid, x_format, x_transform, x_tooltip_label, x_tooltip_format, 
                            y_label, y_variable, y_datatype,  y_zero, y_grid, y_format, y_transform, y_tooltip_label, y_tooltip_format,
                           c_label, c_variable, c_datatype, c_tooltip_label, c_tooltip_format):
    chart = alt.Chart(data).mark_area().encode(
        x=alt.X("{}:{}".format(x_variable, x_datatype), axis=alt.Axis(title=x_label, grid=x_grid, format=x_format), scale=alt.Scale(zero=x_zero)),
        y=alt.Y("{}:{}".format(y_variable, y_datatype), axis=alt.Axis(title=y_label, grid=y_grid, format=y_format), scale=alt.Scale(zero=y_zero)),
        color=alt.Color("{}:{}".format(c_variable, c_datatype), legend=alt.Legend(title=c_label)),
        tooltip=[alt.Tooltip("{}:{}".format(x_variable, x_datatype), title=x_tooltip_label, format=x_tooltip_format),
            alt.Tooltip("{}:{}".format(y_variable, y_datatype), title=y_tooltip_label, format=y_tooltip_format),
            alt.Tooltip("{}:{}".format(c_variable, c_datatype), title=c_tooltip_label, format=c_tooltip_format)]
           ).properties(
        width=800,
        height=500)
    
    if x_transform:
        chart = chart.transform_calculate(x=x_transform)
    
    if y_transform:
        chart = chart.transform_calculate(y=y_transform)
    
    display(chart)
    
    return chart

In [4]:
children = [widgets.Output() for _ in range(4)]
control = widgets.HBox(children[:-1]) 
display(control)
output = widgets.VBox([children[-1]])
display(output)

w = interactive(make_stacked_area_chart,  
             {'manual': True},
             x_label = 'Year', x_variable = 'year(x)', x_datatype = ['T', 'O', 'N', 'Q', 'G'],  x_zero=False,
                x_grid=False, x_format = '', x_transform = '', x_tooltip_label='Year', x_tooltip_format='',
             y_label = 'Greenhouse Gas Emissions (tonnes CO2e)', y_variable ='y', y_datatype = ['Q', 'O', 'N', 'T', 'G'], y_zero=False,
                y_grid=True, y_format = 's', y_transform = 'datum.y*1000000', y_tooltip_label='Emissions (tonnes CO2e)', y_tooltip_format='.3s',
             c_label = 'Sector', c_variable = 'c', c_datatype = ['N', 'O', 'Q', 'T', 'G'], c_tooltip_label='Sector', c_tooltip_format='')
    
with children[0]:
    display(VBox(w.children[:9]))
    
with children[1]:
    display(VBox(w.children[9:18]))
    
with children[2]:
    display(VBox(w.children[18:-1]))
    
with children[3]:
    display(w.children[-1])

# For data types, refer to the documentation at https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types
# The make_stacked_area_chart function only includes transform_calculate for now
# To add more transformations, refer to the documentation at https://altair-viz.github.io/user_guide/transform/index.html
# After filling in all the parameters, click the 'Run Interact' button. An empty chart indicates an error has occurred. 
# In that case, please double check if x_variable, y_variable, and c_variable have been set to the correct values. 